### Read Credentials From File

In [1]:
import configparser
configs = configparser.ConfigParser()
configs.read("cred.ini")
access_key = configs.get("Info", "ACCESS_KEY")
secret_key = configs.get("Info", "SECRET_KEY")
bucket_name = configs.get("Info","BUCKET")
host_region = configs.get("Info","HOST_REGION")


### Initial Connection to S3

In [2]:
# from boto.s3.connection import S3Connection
from botocore.config import Config
import boto3

# conn = S3Connection(access_key, secret_key,host=host_region,proxy='127.0.0.1'.encode(),proxy_port='3128')
conn = boto3.client('s3',aws_access_key_id=access_key,
                    aws_secret_access_key=secret_key,
                    region_name='us-east-2'
                    #,config=Config(proxies={'https': 'localhost:3128'})
                    )


### Get List of Files from bucket (limited right now to the page of 1000)

In [3]:
file_list = list(conn.list_objects(Bucket=bucket_name)['Contents'])
len(file_list)

1000

### Execute Algorithm on 1 file

In [4]:
from StringCounter import *
import time
t1 = time.time()
obj = conn.get_object(Bucket=bucket_name, Key=file_list[0]['Key'])
content = extractFileContent(obj)
cnts = countContent(content,8,30)
t2 = time.time()
delta = t2-t1
print("Time elasped: ", delta)
print("Estimated Time: ", delta*100000)
print("Total strings found:", len(cnts))

Time elasped:  27.776870250701904
Estimated Time:  2777687.0250701904
Total strings found: 17065716


In [5]:
import operator
cnts2 = {k: v for k, v in cnts.items() if v > 1}
cnts_sort = sorted(cnts2.items(), key=operator.itemgetter(1),reverse=True)
final = cnts_sort[:10000]

In [7]:
final

[('////////', 408),
 ('/////////', 402),
 ('//////////', 396),
 ('///////////', 390),
 ('////////////', 384),
 ('/////////////', 378),
 ('//////////////', 372),
 ('///////////////', 366),
 ('////////////////', 360),
 ('/////////////////', 354),
 ('//////////////////', 348),
 ('///////////////////', 342),
 ('////////////////////', 336),
 ('/////////////////////', 330),
 ('//////////////////////', 324),
 ('///////////////////////', 318),
 ('////////////////////////', 312),
 ('/////////////////////////', 306),
 ('//////////////////////////', 300),
 ('///////////////////////////', 294),
 ('////////////////////////////', 288),
 ('/////////////////////////////', 282),
 ('22222222', 172),
 ('ThemePan', 171),
 ('ThemePane', 171),
 ('ThemePanel', 171),
 ('hemePane', 171),
 ('hemePanel', 171),
 ('emePanel', 171),
 ('222222222', 168),
 ('2222222222', 164),
 ('22222222222', 160),
 ('222222222222', 156),
 ('2222222222222', 152),
 ('22222222222222', 148),
 ('ThemePanelM', 145),
 ('hemePanelM', 145),

### Below is scratch

In [5]:
import operator

cnts_sort = sorted(cnts2.items(), key=operator.itemgetter(1),reverse=True)
len(cnts_sort)
print(cnts_sort[:10000])
v = cnts_sort[:1][0][0]

[('////////', 408), ('/////////', 402), ('//////////', 396), ('///////////', 390), ('////////////', 384), ('/////////////', 378), ('//////////////', 372), ('///////////////', 366), ('////////////////', 360), ('/////////////////', 354), ('//////////////////', 348), ('///////////////////', 342), ('////////////////////', 336), ('/////////////////////', 330), ('//////////////////////', 324), ('///////////////////////', 318), ('////////////////////////', 312), ('/////////////////////////', 306), ('//////////////////////////', 300), ('///////////////////////////', 294), ('////////////////////////////', 288), ('/////////////////////////////', 282), ('22222222', 172), ('ThemePan', 171), ('ThemePane', 171), ('ThemePanel', 171), ('hemePane', 171), ('hemePanel', 171), ('emePanel', 171), ('222222222', 168), ('2222222222', 164), ('22222222222', 160), ('222222222222', 156), ('2222222222222', 152), ('22222222222222', 148), ('ThemePanelM', 145), ('hemePanelM', 145), ('emePanelM', 145), ('mePanelM', 14

[('////////', 378), ('/////////', 372), ('//////////', 366), ('///////////', 360), ('////////////', 354), ('/////////////', 348), ('//////////////', 342), ('///////////////', 336), ('////////////////', 330), ('/////////////////', 324), ('//////////////////', 318), ('///////////////////', 312), ('////////////////////', 306), ('/////////////////////', 300), ('//////////////////////', 294), ('///////////////////////', 288), ('////////////////////////', 282), ('/////////////////////////', 276), ('//////////////////////////', 270), ('///////////////////////////', 264), ('////////////////////////////', 258), ('/////////////////////////////', 252), ('22222222', 172), ('ThemePan', 171), ('ThemePane', 171), ('ThemePanel', 171), ('hemePane', 171), ('hemePanel', 171), ('emePanel', 171), ('222222222', 168), ('2222222222', 164), ('22222222222', 160), ('222222222222', 156), ('2222222222222', 152), ('22222222222222', 148), ('ThemePanelM', 145), ('hemePanelM', 145), ('emePanelM', 145), ('mePanelM', 14

In [5]:
import unicodedata
def remove_control_characters(s):
    return "".join(ch for ch in s if unicodedata.category(ch)[0]!="C")

In [10]:
v.replace("\\x00","")

''

In [25]:
chr(0)

'\x00'

In [21]:
def strip_control_characters(input):
    if input:

        import re

        # unicode invalid characters
        RE_XML_ILLEGAL = u'([\u0000-\u0008\u000b-\u000c\u000e-\u001f\ufffe-\uffff])' + \
                         u'|' + \
                         u'([%s-%s][^%s-%s])|([^%s-%s][%s-%s])|([%s-%s]$)|(^[%s-%s])' % \
                          (chr(0xd800),chr(0xdbff),chr(0xdc00),chr(0xdfff),
                           chr(0xd800),chr(0xdbff),chr(0xdc00),chr(0xdfff),
                           chr(0xd800),chr(0xdbff),chr(0xdc00),chr(0xdfff),
                           )
        input = re.sub(RE_XML_ILLEGAL, "", input)
        # ascii control characters
        input = re.sub(r"[\x01-\x1F\x7F]", "", input)

    return input

In [16]:
 from sys import getsizeof
 getsizeof(cnts_sort)

4208808

In [6]:
conn.download_file(bucket_name, '00000f461abe9dfecac5558777ed3a1be4d014f57a6109a6519c73055e38b818_2053519.tar', '00000f461abe9dfecac5558777ed3a1be4d014f57a6109a6519c73055e38b818_2053519.tar')